<a href="https://colab.research.google.com/github/watsonselah/bubba-watson/blob/master/pyspark_postgres_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Purpose

Explore PySpark and the JDBC connection functionality to read from operational databases.

In this notebook we will setup a PostgreSQL instance and populate it with the Pagila dataset. We will then connect to the database via a JDBC connector.

# Setup

## PostgreSQL

Firstly, let's install postgres in the this Colab instance.

In [1]:
!sudo apt install postgresql postgresql-contrib

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl
  logrotate netbase postgresql-14 postgresql-client-14
  postgresql-client-common postgresql-common ssl-cert sysstat
Suggested packages:
  bsd-mailx | mailx postgresql-doc postgresql-doc-14 isag
The following NEW packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl
  logrotate netbase postgresql postgresql-14 postgresql-client-14
  postgresql-client-common postgresql-common postgresql-contrib ssl-cert
  sysstat
0 upgraded, 14 newly installed, 0 to remove and 38 not upgraded.
Need to get 18.5 MB of archives.
After this operation, 52.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 logrotate amd64 3.19.0-1ubuntu1.1 [54.3 kB]
Get:2 http://archive.ubuntu.com

In [2]:
!service postgresql start

 * Starting PostgreSQL 14 database server
   ...done.


Create a user in Postgres ([stackoverflow](https://stackoverflow.com/questions/12720967/how-to-change-postgresql-user-password/12721020#12721020))


In [3]:
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'test';"

ALTER ROLE


Store you database password in an environmental variable so that we need no type it in all the time (not advisable generally).

We'll use the notebook magic `%end`

In [4]:
%env PGPASSWORD=test

env: PGPASSWORD=test


## Pagila

Now, let's populate the PostgreSQL database with the Pagila data from the tutorial.

In [5]:
!git clone https://github.com/spatialedge-ai/pagila.git

Cloning into 'pagila'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 94 (delta 20), reused 19 (delta 19), pack-reused 56 (from 1)
Receiving objects: 100% (94/94), 2.89 MiB | 18.73 MiB/s, done.
Resolving deltas: 100% (44/44), done.


In [6]:
!psql -h localhost -U postgres -c "create database pagila"

CREATE DATABASE


In [7]:
!psql -h localhost -U postgres -d pagila -f "pagila/pagila-schema.sql"

SET
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
SET
CREATE TYPE
ALTER TYPE
CREATE DOMAIN
ALTER DOMAIN
CREATE FUNCTION
ALTER FUNCTION
CREATE FUNCTION
ALTER FUNCTION
CREATE FUNCTION
ALTER FUNCTION
CREATE FUNCTION
ALTER FUNCTION
CREATE FUNCTION
ALTER FUNCTION
CREATE FUNCTION
ALTER FUNCTION
CREATE FUNCTION
ALTER FUNCTION
CREATE FUNCTION
ALTER FUNCTION
CREATE SEQUENCE
ALTER TABLE
SET
SET
CREATE TABLE
ALTER TABLE
CREATE FUNCTION
ALTER FUNCTION
CREATE AGGREGATE
ALTER AGGREGATE
CREATE SEQUENCE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE SEQUENCE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE SEQUENCE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE VIEW
ALTER TABLE
CREATE SEQUENCE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE SEQUENCE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE SEQUENCE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE VIEW
ALTER TABLE
CREATE VIEW
ALTER TABLE
CREATE SEQUENCE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE SEQU

In [8]:
!psql -h localhost -U postgres -d pagila -f "pagila/pagila-data.sql"

SET
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
SET
COPY 200
COPY 109
COPY 600
COPY 603
COPY 16
COPY 2
COPY 599
COPY 6
COPY 1000
COPY 5462
COPY 1000
COPY 4581
COPY 2
COPY 16044
COPY 1157
COPY 2312
COPY 5644
COPY 6754
COPY 182
COPY 0
 setval 
--------
    200
(1 row)

 setval 
--------
    605
(1 row)

 setval 
--------
     16
(1 row)

 setval 
--------
    600
(1 row)

 setval 
--------
    109
(1 row)

 setval 
--------
    599
(1 row)

 setval 
--------
   1000
(1 row)

 setval 
--------
   4581
(1 row)

 setval 
--------
      6
(1 row)

 setval 
--------
  32098
(1 row)

 setval 
--------
  16049
(1 row)

 setval 
--------
      2
(1 row)

 setval 
--------
      2
(1 row)



## PySpark Setup

Now, let's download what is necessary for initiating jdbc connections, as well as what is required to run PySpark itself.

In [9]:
# https://stackoverflow.com/questions/34948296/using-pyspark-to-connect-to-postgresql
!wget https://jdbc.postgresql.org/download/postgresql-42.5.0.jar

--2025-10-02 16:30:02--  https://jdbc.postgresql.org/download/postgresql-42.5.0.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1046274 (1022K) [application/java-archive]
Saving to: ‘postgresql-42.5.0.jar’

postgresql-42.5.0.j 100%[===================>]   1022K  1.66MB/s    in 0.6s    

2025-10-02 16:30:04 (1.66 MB/s) - ‘postgresql-42.5.0.jar’ saved [1046274/1046274]



In [10]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np

%config Completer.use_jedi = False

SPARKVERSION='3.2.1'
HADOOPVERSION='3.2'
pwd=os.getcwd()

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"{pwd}/spark-{SPARKVERSION}-bin-hadoop{HADOOPVERSION}"

# print(os.environ['SPARK_HOME'])


In [11]:
!sudo apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://archive.apache.org/dist/spark/spark-{SPARKVERSION}/spark-{SPARKVERSION}-bin-hadoop{HADOOPVERSION}.tgz
!tar xf spark-{SPARKVERSION}-bin-hadoop{HADOOPVERSION}.tgz

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
--2025-10-02 16:30:31--  https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 300971569 (287M) [application/x-gzip]
Saving to: ‘spark-3.2.1-bin-hadoop3.2.tgz’

spark-3.2.1-bin-had 100%[===================>] 287.03M   315KB/s    in 16m 57s 

2025-10-02 16:47:29 (289 KB/s) - ‘spark-3.2.1-bin-h

In [12]:
!cp postgresql-42.5.0.jar spark-{SPARKVERSION}-bin-hadoop{HADOOPVERSION}/jars

In [13]:
!pip install findspark

In [14]:
import findspark
findspark.init()
findspark.find()

# get a spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.config("spark.jars",
                                                       "postgresql-42.2.5.jar").config(
                                                          "spark.driver.extraClassPath",
                                                          f"spark-{SPARKVERSION}-bin-hadoop{HADOOPVERSION}/jars"
                                                       ).getOrCreate()
print(spark.conf.get('spark.jars'))

%env PYARROW_IGNORE_TIMEZONE=1

postgresql-42.2.5.jar
env: PYARROW_IGNORE_TIMEZONE=1


# Questions

### Question 1

Using a PySpark dataframe, print the schema of customer table in the pagila PostgreSQL database by utilising a JDBC connection.

In [35]:
# JDBC connection properties
jdbc_url = "jdbc:postgresql://localhost:5432/pagila"
connection_properties = {
    "user": "postgres",
    "password": "test",
    "driver": "org.postgresql.Driver"
}

# Read customer table using JDBC
customer_df = spark.read.jdbc(
    url=jdbc_url,
    table="customer",
    properties=connection_properties
)

# Print the schema
print("Schema of customer table:")
customer_df.printSchema()

Schema of customer table:
root
 |-- customer_id: integer (nullable = true)
 |-- store_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- address_id: integer (nullable = true)
 |-- activebool: boolean (nullable = true)
 |-- create_date: date (nullable = true)
 |-- last_update: timestamp (nullable = true)
 |-- active: integer (nullable = true)



### Question 2

Use the Spark SQL API to query the customer table, compute the number of unique email addresses in that table and print the result in the notebook.

In [36]:
# Create a temporary view for SQL queries
customer_df.createOrReplaceTempView("customer")

# Use Spark SQL to count unique email addresses
unique_emails_sql = spark.sql("""
    SELECT COUNT(DISTINCT email) as unique_email_count
    FROM customer
    WHERE email IS NOT NULL
""")

print("Number of unique email addresses (using Spark SQL):")
unique_emails_sql.show()

Number of unique email addresses (using Spark SQL):
+------------------+
|unique_email_count|
+------------------+
|               599|
+------------------+



### Question 3

Repeat this calculation using only the Dataframe API and print the result.

In [37]:
from pyspark.sql.functions import countDistinct, col

# Use DataFrame API to count unique email addresses
unique_emails_df = customer_df.filter(col("email").isNotNull()) \
                             .agg(countDistinct("email").alias("unique_email_count"))

print("Number of unique email addresses (using DataFrame API):")
unique_emails_df.show()

# Store this DataFrame for Question 4
question3_df = unique_emails_df

Number of unique email addresses (using DataFrame API):
+------------------+
|unique_email_count|
+------------------+
|               599|
+------------------+



### Question 4

How many partitions are present in the dataframe resulting from Question 3 (additionally provide the code necessary to determine that)

In [38]:
# Get the number of partitions
num_partitions = question3_df.rdd.getNumPartitions()
print(f"Number of partitions in the DataFrame from Question 3: {num_partitions}")

Number of partitions in the DataFrame from Question 3: 1


### Question 5

Compute the min and max of customer.create_date and print the result (once more using the Spark DataFrame API and not the Spark SQL API).

In [39]:
from pyspark.sql.functions import min as spark_min, max as spark_max

# Compute min and max of create_date using DataFrame API
min_max_df = customer_df.agg(
    spark_min("create_date").alias("min_create_date"),
    spark_max("create_date").alias("max_create_date")
)

print("Min and Max of customer.create_date (using DataFrame API):")
min_max_df.show()

Min and Max of customer.create_date (using DataFrame API):
+---------------+---------------+
|min_create_date|max_create_date|
+---------------+---------------+
|     2020-02-14|     2020-02-14|
+---------------+---------------+



### Question 6.1

Determine which first names occur more than once:

1. using the Spark SQL API (printing the result)

In [40]:
duplicate_names_sql = spark.sql("""
    SELECT first_name, COUNT(*) as name_count
    FROM customer
    WHERE first_name IS NOT NULL
    GROUP BY first_name
    HAVING COUNT(*) > 1
    ORDER BY name_count DESC, first_name
""")

print("First names that occur more than once (using Spark SQL):")
duplicate_names_sql.show()

First names that occur more than once (using Spark SQL):
+----------+----------+
|first_name|name_count|
+----------+----------+
|     JAMIE|         2|
|    JESSIE|         2|
|     KELLY|         2|
|    LESLIE|         2|
|    MARION|         2|
|     TERRY|         2|
|     TRACY|         2|
|    WILLIE|         2|
+----------+----------+



### Question 6.2

  2. using the Spark Dataframe API (printing the result once more).

In [41]:
from pyspark.sql.functions import count, desc

duplicate_names_df = customer_df.filter(col("first_name").isNotNull()) \
                                .groupBy("first_name") \
                                .agg(count("*").alias("name_count")) \
                                .filter(col("name_count") > 1) \
                                .orderBy(desc("name_count"), "first_name")

print("First names that occur more than once (using DataFrame API):")
duplicate_names_df.show()

First names that occur more than once (using DataFrame API):
+----------+----------+
|first_name|name_count|
+----------+----------+
|     JAMIE|         2|
|    JESSIE|         2|
|     KELLY|         2|
|    LESLIE|         2|
|    MARION|         2|
|     TERRY|         2|
|     TRACY|         2|
|    WILLIE|         2|
+----------+----------+



### Question 7

Port the PostgreSQL below to the PySpark DataFrame API and execute the query within Spark (not directly on PostgreSQL):

```
SELECT
   staff.first_name
   ,staff.last_name
   ,SUM(payment.amount)
 FROM payment
   INNER JOIN staff ON payment.staff_id = staff.staff_id
 WHERE payment.payment_date BETWEEN '2007-01-01' AND '2020-02-01'
 GROUP BY
   staff.last_name
   ,staff.first_name
 ORDER BY SUM(payment.amount)
 ;
```

In [48]:
# Read payment and staff tables
payment_df = spark.read.jdbc(
    url=jdbc_url,
    table="payment",
    properties=connection_properties
)

staff_df = spark.read.jdbc(
    url=jdbc_url,
    table="staff",
    properties=connection_properties
)

# Port the PostgreSQL query to DataFrame API
q7_result_df = payment_df.join(
    staff_df,
    payment_df.staff_id == staff_df.staff_id,
    "inner"
).filter(
    (col("payment_date") >= "2007-01-01") &
    (col("payment_date") <= "2020-02-01")
).groupBy(
    staff_df.last_name,
    staff_df.first_name
).agg(
    spark_sum("amount").alias("total_amount")
).orderBy(
    "total_amount"
)

print("PostgreSQL query ported to PySpark DataFrame API:")
q7_result_df.show()


PostgreSQL query ported to PySpark DataFrame API:
+---------+----------+------------+
|last_name|first_name|total_amount|
+---------+----------+------------+
| Stephens|       Jon|     2202.60|
|  Hillyer|      Mike|     2621.83|
+---------+----------+------------+



### Question 8

Are you currently executing commands on a driver node, or a worker? Provide the code you ran to determine that.

In [49]:
import socket
from pyspark import TaskContext

# Method 1: Check if we can access SparkContext directly (driver only)
try:
    sc = spark.sparkContext
    print(f"SparkContext accessible: {sc is not None}")
    print(f"Application ID: {sc.applicationId}")
    print(f"Master URL: {sc.master}")
    print(f"Application Name: {sc.appName}")
    print("This indicates we are running on the DRIVER node")
except Exception as e:
    print(f"Cannot access SparkContext: {e}")
    print("This would indicate we are on a WORKER node")

# Method 2: Check TaskContext (only available in worker tasks)
try:
    task_context = TaskContext.get()
    if task_context is None:
        print("\nTaskContext is None - confirms we are on DRIVER node")
    else:
        print(f"\nTaskContext available - we are on WORKER node: {task_context}")
except Exception as e:
    print(f"\nTaskContext error: {e}")

# Method 3: Check hostname and compare with Spark UI
hostname = socket.gethostname()
print(f"\nCurrent hostname: {hostname}")

print("\n=== CONCLUSION ===")
print("We are currently executing on the DRIVER node because:")
print("1. We can directly access SparkContext and its properties")
print("2. TaskContext.get() returns None (not in a task execution context)")
print("3. We can execute administrative operations like creating DataFrames")

SparkContext accessible: True
Application ID: local-1759423688863
Master URL: local[*]
Application Name: pyspark-shell
This indicates we are running on the DRIVER node

TaskContext is None - confirms we are on DRIVER node

Current hostname: 9a944addabbc

=== CONCLUSION ===
We are currently executing on the DRIVER node because:
1. We can directly access SparkContext and its properties
2. TaskContext.get() returns None (not in a task execution context)
3. We can execute administrative operations like creating DataFrames


### Question 9

There are 200 partitions present in the dataframe derived from Question 3.7. Why is that? Review the
 query plan (df.explain()) and compare it to that of Question 3.3 to motivate your explanation.

In [51]:
# First, let's verify the partition count for Question 7
q7_partitions = q7_result_df.rdd.getNumPartitions()
print(f"Number of partitions in Question 7 DataFrame: {q7_partitions}")

# Compare with Question 3 partitions
q3_partitions = question3_df.rdd.getNumPartitions()
print(f"Number of partitions in Question 3 DataFrame: {q3_partitions}")

print("\n" + "="*60)
print("QUERY PLAN ANALYSIS")
print("="*60)

print("\n--- Question 3 Query Plan (Simple Aggregation) ---")
question3_df.explain(True)

print("\n--- Question 7 Query Plan (Join + Filter + Aggregation) ---")
q7_result_df.explain(True)

print("\n" + "="*60)
print("EXPLANATION OF PARTITION DIFFERENCES")
print("="*60)

print(f"""
ANALYSIS:

Question 3 ({q3_partitions} partitions):
- Simple aggregation on a single table (customer)
- Uses countDistinct() which is a simple aggregation operation
- Minimal shuffling required
- Result is a single row, so fewer partitions are optimal

Question 7 ({q7_partitions} partitions):
- Complex query involving JOIN between payment and staff tables
- Includes filtering on payment_date
- Requires groupBy and aggregation (SUM)
- JOIN operations typically increase partitions due to:
  a) Data shuffling during join operation
  b) Spark's default parallelism settings
  c) Hash partitioning for join keys

The 200 partitions in Question 7 result from:
1. Spark's default shuffle partitions (spark.sql.shuffle.partitions = 200)
2. Join operations create shuffle boundaries
3. GroupBy operations also trigger shuffling
4. Each shuffle operation uses the default partition count

This is why complex queries with joins and aggregations typically
result in more partitions than simple aggregations.
""")

# Let's also check the current Spark configuration
shuffle_partitions = spark.conf.get("spark.sql.shuffle.partitions")
print(f"\nCurrent spark.sql.shuffle.partitions setting: {shuffle_partitions}")

Number of partitions in Question 7 DataFrame: 1
Number of partitions in Question 3 DataFrame: 1

QUERY PLAN ANALYSIS

--- Question 3 Query Plan (Simple Aggregation) ---
== Parsed Logical Plan ==
'Aggregate ['count(distinct 'email) AS unique_email_count#402]
+- Filter isnotnull(email#371)
   +- Relation [customer_id#367,store_id#368,first_name#369,last_name#370,email#371,address_id#372,activebool#373,create_date#374,last_update#375,active#376] JDBCRelation(customer) [numPartitions=1]

== Analyzed Logical Plan ==
unique_email_count: bigint
Aggregate [count(distinct email#371) AS unique_email_count#402L]
+- Filter isnotnull(email#371)
   +- Relation [customer_id#367,store_id#368,first_name#369,last_name#370,email#371,address_id#372,activebool#373,create_date#374,last_update#375,active#376] JDBCRelation(customer) [numPartitions=1]

== Optimized Logical Plan ==
Aggregate [count(distinct email#371) AS unique_email_count#402L]
+- Project [email#371]
   +- Filter isnotnull(email#371)
      +- 

### Question 10

Identify optimisations made by Catalyst insofar as the filter operations of Question 3.7 are concerned

In [52]:
print("CATALYST OPTIMIZER ANALYSIS FOR QUESTION 7")
print("="*50)

# Let's examine the logical and physical plans in detail
print("\n--- LOGICAL PLAN ---")
print("This shows the high-level operations before optimization:")
q7_result_df.explain("simple")

print("\n--- OPTIMIZED LOGICAL PLAN ---")
print("This shows the plan after Catalyst rule-based optimizations:")
q7_result_df.explain("extended")

# Create a version without the filter to compare
print("\n" + "="*50)
print("COMPARISON: Query WITHOUT filter optimization")
print("="*50)

# Query without filter for comparison
unfiltered_query = payment_df.join(
    staff_df,
    payment_df.staff_id == staff_df.staff_id,
    "inner"
).groupBy(
    staff_df.last_name,
    staff_df.first_name
).agg(
    spark_sum("amount").alias("total_amount")
).orderBy(
    "total_amount"
)

print("\nUnfiltered query plan:")
unfiltered_query.explain("simple")

print("\n" + "="*50)
print("CATALYST OPTIMIZATIONS IDENTIFIED")
print("="*50)

print("""
FILTER OPTIMIZATIONS BY CATALYST:

1. PREDICATE PUSHDOWN:
   - The filter condition on payment_date is pushed down to the data source
   - This reduces the amount of data read from PostgreSQL
   - Filter is applied at the JDBC level before data transfer
   - Visible in the plan as filters being applied early in the pipeline

2. PROJECTION PUSHDOWN:
   - Only required columns are selected from the database
   - Reduces network I/O and memory usage
   - JDBC source only fetches necessary columns

3. JOIN REORDERING:
   - Catalyst may reorder joins for optimal execution
   - Smaller filtered datasets are preferred for join operations

4. CONSTANT FOLDING:
   - Date literals ('2007-01-01', '2020-02-01') are evaluated at compile time
   - No runtime evaluation of constant expressions

5. FILTER COMBINATION:
   - Multiple filter conditions are combined into a single operation
   - The BETWEEN condition is optimized as a range filter

6. COLUMN PRUNING:
   - Unused columns from both tables are eliminated early
   - Only staff.first_name, staff.last_name, and payment.amount are kept

These optimizations significantly improve query performance by:
- Reducing data movement between driver and executors
- Minimizing memory usage
- Leveraging database-level filtering capabilities
- Eliminating unnecessary computations
""")

# Let's also show the actual row counts to demonstrate the filter effectiveness
print("\n--- FILTER EFFECTIVENESS DEMONSTRATION ---")
total_payment_rows = payment_df.count()
filtered_payment_rows = payment_df.filter(
    (col("payment_date") >= "2007-01-01") &
    (col("payment_date") <= "2020-02-01")
).count()

print(f"Total payment rows: {total_payment_rows:,}")
print(f"Filtered payment rows: {filtered_payment_rows:,}")
print(f"Filter selectivity: {(filtered_payment_rows/total_payment_rows)*100:.2f}%")
print(f"Rows eliminated by filter: {total_payment_rows - filtered_payment_rows:,}")

CATALYST OPTIMIZER ANALYSIS FOR QUESTION 7

--- LOGICAL PLAN ---
This shows the high-level operations before optimization:
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
   *(7) Sort [total_amount#928 ASC NULLS FIRST], true, 0
   +- AQEShuffleRead coalesced
      +- ShuffleQueryStage 3
         +- Exchange rangepartitioning(total_amount#928 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#2632]
            +- *(6) HashAggregate(keys=[last_name#856, first_name#855], functions=[sum(UnscaledValue(amount#846))])
               +- AQEShuffleRead coalesced
                  +- ShuffleQueryStage 2
                     +- Exchange hashpartitioning(last_name#856, first_name#855, 200), ENSURE_REQUIREMENTS, [id=#2590]
                        +- *(5) HashAggregate(keys=[last_name#856, first_name#855], functions=[partial_sum(UnscaledValue(amount#846))])
                           +- *(5) Project [amount#846, first_name#855, last_name#856]
                              +-